<a href="https://colab.research.google.com/github/saishshinde15/Langchain_RAG_BOT/blob/main/Learning_Enhanced_RAG_(LE_RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [2]:
pip install langchain_core

In [3]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.5 MB/s eta 0:00:00


In [4]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.0 MB/s eta 0:00:00


In [5]:
import os
import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [6]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [7]:
llm=ChatGroq(model='llama-3.1-8b-instant',temperature=0.5)

In [8]:
def classify_query(query):
    """Classify user query into types for enhanced processing."""
    if "how" in query or "impact" in query:
        return "analytical"
    elif "what" in query or "definition" in query:     #Function to classify the query
        return "factual"
    elif "example" in query or "case" in query:
        return "opinion"
    else:
        return "contextual"

In [9]:
from langchain_core.prompts import ChatPromptTemplate

In [10]:
from langchain_core.output_parsers import StrOutputParser

In [11]:
def enhance_query(query):
    """Enhance the original query using a language model."""
    system = """You a question re-writer that converts an input question to a better version that is optimized
     for vectorstore retrieval. Look at the input and try to reason about the underlying semantic intent / meaning."""
    re_write_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            (
                "human",
                "Here is the initial question: \n\n {question} \n Formulate an improved question.",
            ),
        ]
    )
    question_rewriter = re_write_prompt | llm | StrOutputParser()
    enhanced_query = question_rewriter.invoke({"question": query})
    return enhanced_query

In [12]:
def fetch_documents_from_urls(urls):# dont run
    """Fetch content from provided URLs and return as documents."""
    documents = []
    for url in urls:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            soup = BeautifulSoup(response.text, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)  # Extract text
            documents.append({"text": text, "metadata": {"source": url}})
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch {url}: {e}")
    return documents

In [13]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
url_list = [
    "https://www.cisa.gov/uscert/ncas/current-activity",
    "https://www.kaspersky.com/resource-center/threats",
    "https://www.cyber.gov.au/acsc/view-all-content/publications"
]

In [14]:

urls = [
    "https://www.simplilearn.com/tutorials/cyber-security-tutorial/types-of-cyber-attacks",
    "https://www.fortinet.com/resources/cyberglossary/types-of-cyber-attacks",
    "https://cheatsheetseries.owasp.org/cheatsheets/SQL_Injection_Prevention_Cheat_Sheet.html",
]

In [15]:
loader=WebBaseLoader(urls)

In [16]:
documents = loader.load()

In [17]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)
documents = text_splitter.split_documents(documents)

In [18]:
documents[0]

Document(metadata={'source': 'https://www.simplilearn.com/tutorials/cyber-security-tutorial/types-of-cyber-attacks', 'title': 'Types of Cyber Attacks You Should Be Aware of in 2025', 'description': 'There are many varieties of cyber attacks that happen in the world. If we know the various types of cyberattacks, it becomes easier for us to protect our networks. Know more!', 'language': 'en-US'}, page_content='Types of Cyber Attacks You Should Be Aware of in 2025')

In [19]:
import getpass

inference_api_key = getpass.getpass("Enter your HF Inference API Key:\n\n")

Enter your HF Inference API Key:

··········


In [20]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [21]:
if documents:
    vectorstore = FAISS.from_documents(documents, embeddings)
    retriever = VectorStoreRetriever(vectorstore=vectorstore)
else:
    raise ValueError("No documents were fetched. Please check the URLs.")


In [22]:
memory = ConversationBufferMemory(memory_key="chat_history")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    memory=memory,
)

<ipython-input-22-41a58f208115>:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [ ]:
query = "What are the types of cyber attacks?"
enhanced_query = enhance_query(query)
result = qa_chain({"query": enhanced_query})
print(result["result"])


<ipython-input-23-d30b7348edce>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": enhanced_query})


Your re-formulated question is indeed more specific and granular, making it more suitable for vectorstore retrieval. By including specific keywords, categories, subtypes, and characteristics, you're providing a richer context for the vectorstore to understand the semantic intent and meaning behind the question.

Here are some suggestions to further optimize the question for vectorstore retrieval:

1. **Use more precise keywords**: Instead of using general terms like "cyber attacks," consider using more specific phrases like "types of cyber threats," "cybersecurity attack vectors," or "malware attack classifications."
2. **Add relevant context**: Provide some background information or context that might help the vectorstore understand the scope and relevance of the question. For example, you could mention the industry, organization, or specific systems that the question is related to.
3. **Use a more formal query structure**: Consider using a more structured query format, such as a ques

In [ ]:
## Dont Run below code as CPU useage:Try compressioning the code

In [ ]:
def gather_feedback(response, user_feedback, retriever, query_type, memory):
    """
    Collect user feedback and simulate continuous learning.
    Adjust the query classification or retriever based on feedback.
    """
    print("User Feedback: ", user_feedback)

    # If feedback is irrelevant, simulate adjusting query enhancement or document relevance
    if user_feedback.lower() == "irrelevant":
        print("Adjusting the query classification or retrieval process...")

        # 1. Change the query classification if it misclassified
        if query_type == "factual":
            print("Reclassifying query as 'analytical'.")
            query_type = "analytical"  # Adjust classification for future runs

        # 2. Alternatively, adjust the document retriever for future queries
        print("Increasing importance of document feedback...")
        memory.save_context({"feedback": "irrelevant"}, {"response": response})

    else:
        print("Feedback noted. The system will continue with the current process.")


In [ ]:
# Step 10: Main Function to Handle User Queries
def handle_user_query(query):
    """Process user query, enhance it, retrieve relevant documents, and generate a response."""
    # Classify the query
    query_type = classify_query(query)
    print(f"Query Type: {query_type}")

    # Enhance the query
    enhanced_query = enhance_query(query)
    print(f"Enhanced Query: {enhanced_query}")

    # Retrieve relevant documents
    retrieved_docs = retriever.get_relevant_documents(enhanced_query)
    if retrieved_docs:
        print(f"Retrieved Documents: {retrieved_docs}")
    else:
        print("No documents found for the enhanced query.")
        return

    # Generate response
    response = qa_chain.run(input=enhanced_query)  # Pass enhanced_query as the input
    print(f"Generated Response: {response}")

    # Simulating user feedback for learning
    user_feedback = input("Please provide feedback on the response (relevant/irrelevant): ")
    gather_feedback(response, user_feedback, retriever, query_type, memory)

# Step 11: Example Usage
if __name__ == "__main__":
    user_query = input("Enter your cybersecurity query: ")
    handle_user_query(user_query)

In [ ]:
## Compressed code(Better CPU load)

In [23]:
def run_feedback_loop_simulation(num_iterations=5):
    """Simulates a feedback loop for a QA system."""

    for i in range(num_iterations):
        print(f"\n--- Iteration {i+1} ---")

        # 1. User Query Input (Simulate)
        user_query = input("Enter your query: ")

        # 2. Query Classification
        query_type = classify_query(user_query)
        print(f"Query type classified as: {query_type}")

        # 3. Query Enhancement (Optional, based on query type)
        if query_type == "analytical" or query_type == "factual":
            enhanced_query = enhance_query(user_query)
            print(f"Enhanced Query: {enhanced_query}")
        else:
            enhanced_query = user_query

        # 4. Retrieval & Answer Generation
        result = qa_chain({"query": enhanced_query})
        print(f"Answer:\n{result['result']}")

        # 5. Feedback Collection (Simulate)
        feedback = input("Was the answer helpful? (yes/no): ")

        # 6. Feedback Processing and System Update (Placeholder)
        if feedback.lower() == "no":
            print("System needs improvement based on feedback.")



if __name__ == "__main__":
    run_feedback_loop_simulation()


--- Iteration 1 ---
Enter your query: what is sql
Query type classified as: factual
Enhanced Query: Here's a rewritten version of the question optimized for vectorstore retrieval:

"What is the definition and purpose of SQL, including its syntax, functionality, and applications in database management?"

This rewritten question is more specific and includes additional context that can help vectorstore retrieval algorithms better understand the underlying semantic intent. The added keywords such as "definition", "syntax", "functionality", and "applications" can help improve the accuracy of the retrieval results.

Alternatively, you could also break down the question into sub-questions to further refine the search:

* "What is the definition of SQL?"
* "What is the syntax of SQL?"
* "What are the main functionalities of SQL?"
* "What are some common applications of SQL in database management?"

This approach can help to retrieve more relevant results and provide a better understanding of

<ipython-input-23-fbfaa254b7c6>:22: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": enhanced_query})


Answer:
SQL (Structured Query Language) is a programming language designed for managing and manipulating data stored in relational database management systems (RDBMS). The primary purpose of SQL is to provide a standard way of interacting with databases, allowing users to perform various operations such as creating, modifying, and querying database schema and data.

**Definition and Purpose:**

SQL is a declarative language, meaning that users specify what they want to do with their data, rather than how to do it. This allows SQL to optimize the query execution plan and improve performance. SQL is used for various purposes, including:

* Creating and modifying database schema
* Inserting, updating, and deleting data
* Querying data using SELECT statements
* Managing database security and permissions

**Syntax:**

SQL syntax consists of several components, including:

* Keywords: Such as SELECT, FROM, WHERE, GROUP BY, HAVING, etc.
* Identifiers: Table names, column names, and other data

KeyboardInterrupt: Interrupted by user

In [ ]:
def run_feedback_loop_simulation(num_iterations=5):
    """Simulates a feedback loop for a QA system."""

    for i in range(num_iterations):
        print(f"\n--- Iteration {i+1} ---")

        # 1. User Query Input (Simulate)
        user_query = input("Enter your query: ")

        # 2. Query Classification
        query_type = classify_query(user_query)
        print(f"Query type classified as: {query_type}")

        # 3. Query Enhancement (Optional, based on query type)
        if query_type == "analytical" or query_type == "factual":
            enhanced_query = enhance_query(user_query)
            print(f"Enhanced Query: {enhanced_query}")
        else:
            enhanced_query = user_query

        # 4. Retrieval & Answer Generation
        result = qa_chain({"query": enhanced_query})
        print(f"Answer:\n{result['result']}")

        # 5. Feedback Collection (Simulate)
        feedback = input("Was the answer helpful? (yes/no): ")

        # 6. Feedback Processing and System Update (Placeholder)
        if feedback.lower() == "no":
            why_not_helpful = input("Why was the answer not helpful? ") # Added input for reason
            print(f"User provided reason: {why_not_helpful}") # Print the reason
            print("System needs improvement based on feedback.")
        else:
            print("Feedback noted. The system will continue with the current process.") # Optional positive feedback print

if __name__ == "__main__":
    run_feedback_loop_simulation()


--- Iteration 1 ---
Enter your query: what is sql
Query type classified as: factual
Enhanced Query: Here's a re-formulated version of the question optimized for vector store retrieval:

"What is SQL definition, purpose, and usage in database management systems?"

This improved question:

1. **Adds specificity**: By asking for the "definition, purpose, and usage" of SQL, we're providing more context and making it easier for the vector store to retrieve relevant information.
2. **Uses more precise keywords**: Instead of just asking "what is SQL", we're using more specific keywords like "definition", "purpose", and "usage" to capture the underlying semantic intent.
3. **Expands the scope**: By mentioning "database management systems", we're making it clear that we're interested in SQL in the context of databases, which can help the vector store retrieve more relevant information.

This reformulated question should be more effective at retrieving relevant information from a vector store, 

KeyboardInterrupt: Interrupted by user

In [25]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [26]:
import gradio as gr
import os
import requests
from bs4 import BeautifulSoup
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
import getpass
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

# ... (Your existing code for setup, functions, etc.)

def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""

    # 1. Query Classification
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    # 2. Query Enhancement
    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    # 3. Retrieval & Answer Generation
    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer


def feedback_function(user_query, feedback):
    if feedback.lower() == "yes":
        return "Thank you for your feedback. The process continues."
    else:
        reason = gr.Textbox(label="Reason for irrelevant answer:")
        return f"Why was the answer not helpful? {reason}"

with gr.Blocks() as demo:
    with gr.Row():
      user_query = gr.Textbox(label="Enter your query")
      submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    with gr.Row():
      feedback_button_yes = gr.Button("Yes")
      feedback_button_no = gr.Button("No")

    feedback_textbox = gr.Textbox(label = "Feedback")

    def submit_event(user_query):
      return run_feedback_loop_simulation(user_query), ""

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer,feedback_textbox])

    def handle_feedback(answer, feedback_val):
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(value="")
        else:
            return "Please provide a reason.", gr.update(visible=True)

    feedback_button_yes.click(fn=handle_feedback, inputs=[answer,feedback_button_yes], outputs=[feedback_textbox, feedback_textbox])
    feedback_button_no.click(fn=handle_feedback, inputs=[answer,feedback_button_no], outputs=[feedback_textbox, feedback_textbox])


demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2dfbf168f0369b62c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Intial code
import gradio as gr

# ... (Your existing code for setup, functions, etc.)

def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""

    # 1. Query Classification
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    # 2. Query Enhancement
    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    # 3. Retrieval & Answer Generation
    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer


def feedback_function(user_query, feedback, reason=""):
    if feedback.lower() == "yes":
        return "Thank you for your feedback. The process continues.", gr.update(visible=False), ""
    else:
        return "Why was the answer not helpful?", gr.update(visible=True), ""

with gr.Blocks() as demo:
    with gr.Row():
        user_query = gr.Textbox(label="Enter your query")
        submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    with gr.Row():
        feedback_button_yes = gr.Button("Yes")
        feedback_button_no = gr.Button("No")

    feedback_textbox = gr.Textbox(label="Feedback", visible=False)
    reason_textbox = gr.Textbox(label="Reason", visible=False)


    def submit_event(user_query):
        return run_feedback_loop_simulation(user_query), "", gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox, feedback_textbox])

    def handle_feedback(answer, feedback_val, reason=""):
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(visible=False), gr.update(visible=False), ""
        else:
            return "Please provide a reason:", gr.update(visible=True), gr.update(visible=True), ""

    feedback_button_yes.click(fn=handle_feedback, inputs=[answer, feedback_button_yes], outputs=[feedback_textbox, reason_textbox, feedback_textbox, answer])
    feedback_button_no.click(fn=handle_feedback, inputs=[answer, feedback_button_no], outputs=[feedback_textbox, reason_textbox, feedback_textbox, answer])


demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://42e342101dcfb2bab1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# iteration 1(Buttons not working)
import gradio as gr


def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""

    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer

with gr.Blocks() as demo:
    with gr.Row():
        user_query = gr.Textbox(label="Enter your query")
        submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    feedback_textbox = gr.Textbox(label="Feedback", visible=False)
    reason_textbox = gr.Textbox(label="Reason", visible=False)

    with gr.Row():
        feedback_button_yes = gr.Button("Yes")
        feedback_button_no = gr.Button("No")

    def submit_event(user_query):
        return run_feedback_loop_simulation(user_query), gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox])

    def handle_feedback(answer, feedback_val, reason=""):
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(visible=False), gr.update(visible=False)
        else:
            return "Please provide a reason:", gr.update(visible=True), gr.update(visible=True)

    feedback_button_yes.click(fn=handle_feedback, inputs=[answer, feedback_button_yes, reason_textbox], outputs=[feedback_textbox, reason_textbox, feedback_textbox])
    feedback_button_no.click(fn=handle_feedback, inputs=[answer, feedback_button_no, reason_textbox], outputs=[feedback_textbox, reason_textbox, feedback_textbox])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://215f1c812ab1817919.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# iteration 2(reason space not working)
import gradio as gr


def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer

with gr.Blocks() as demo:
    with gr.Row():
        user_query = gr.Textbox(label="Enter your query")
        submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    feedback_textbox = gr.Textbox(label="Feedback", visible=False)
    reason_textbox = gr.Textbox(label="Reason", visible=False)

    with gr.Row():
        feedback_button_yes = gr.Button("Yes")
        feedback_button_no = gr.Button("No")

    def submit_event(user_query):
        # Reset the visibility of feedback fields when a new query is submitted
        return run_feedback_loop_simulation(user_query), gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox])

    def handle_feedback(answer, feedback_val):
        # Display a thank you message if feedback is "Yes" or prompt for a reason if "No"
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(visible=False), gr.update(visible=False)
        else:
            return "Please provide a reason:", gr.update(visible=True), gr.update(visible=True)

    # Use lambda functions to pass "Yes" or "No" as feedback values
    feedback_button_yes.click(fn=lambda answer: handle_feedback(answer, "Yes"), inputs=answer, outputs=[feedback_textbox, reason_textbox, feedback_textbox])
    feedback_button_no.click(fn=lambda answer: handle_feedback(answer, "No"), inputs=answer, outputs=[feedback_textbox, reason_textbox, feedback_textbox])

demo.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1ca9abf8f150f478b2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# iteration 3(No submit button)
import gradio as gr


def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer

with gr.Blocks() as demo:
    with gr.Row():
        user_query = gr.Textbox(label="Enter your query")
        submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    feedback_textbox = gr.Textbox(label="Feedback", visible=False)
    reason_textbox = gr.Textbox(label="Reason", visible=False, interactive=True)  # Set interactive=True for typing

    with gr.Row():
        feedback_button_yes = gr.Button("Yes")
        feedback_button_no = gr.Button("No")

    def submit_event(user_query):
        # Reset the visibility of feedback fields when a new query is submitted
        return run_feedback_loop_simulation(user_query), gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox])

    def handle_feedback(answer, feedback_val):
        # Display a thank you message if feedback is "Yes" or prompt for a reason if "No"
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(visible=False), gr.update(visible=False)
        else:
            return "Please provide a reason:", gr.update(visible=True), gr.update(visible=True)

    # Use lambda functions to pass "Yes" or "No" as feedback values
    feedback_button_yes.click(fn=lambda answer: handle_feedback(answer, "Yes"), inputs=answer, outputs=[feedback_textbox, reason_textbox, feedback_textbox])
    feedback_button_no.click(fn=lambda answer: handle_feedback(answer, "No"), inputs=answer, outputs=[feedback_textbox, reason_textbox, feedback_textbox])

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c76db17241769eda0c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# all working properly

import gradio as gr



def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer

with gr.Blocks() as demo:
    with gr.Row():
        user_query = gr.Textbox(label="Enter your query")
        submit = gr.Button("Submit")

    answer = gr.Textbox(label="Answer")
    feedback_textbox = gr.Textbox(label="Feedback", visible=False)
    reason_textbox = gr.Textbox(label="Reason", visible=False, interactive=True)
    submit_reason_button = gr.Button("Submit Reason", visible=False)  # Add a "Submit Reason" button

    with gr.Row():
        feedback_button_yes = gr.Button("Yes")
        feedback_button_no = gr.Button("No")

    def submit_event(user_query):
        # Reset the visibility of feedback fields when a new query is submitted
        return run_feedback_loop_simulation(user_query), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox, submit_reason_button])

    def handle_feedback(answer, feedback_val):
        # Display a thank you message if feedback is "Yes" or prompt for a reason if "No"
        if feedback_val == "Yes":
            return "Thank you for your feedback. The process continues.", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
        else:
            return "Please provide a reason:", gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)

    # Use lambda functions to pass "Yes" or "No" as feedback values
    feedback_button_yes.click(fn=lambda answer: handle_feedback(answer, "Yes"), inputs=answer, outputs=[feedback_textbox, reason_textbox, submit_reason_button])
    feedback_button_no.click(fn=lambda answer: handle_feedback(answer, "No"), inputs=answer, outputs=[feedback_textbox, reason_textbox, submit_reason_button])

    def submit_reason(reason):
        # Process the submitted reason (e.g., log it, save it)
        print(f"User reason: {reason}")
        return "Thank you for your reason.", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    # Handle the reason submission
    submit_reason_button.click(fn=submit_reason, inputs=reason_textbox, outputs=[feedback_textbox, reason_textbox, submit_reason_button])

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03f90251ec040f3f63.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
import gradio as gr


def run_feedback_loop_simulation(user_query):
    """Simulates a feedback loop for a QA system."""
    query_type = classify_query(user_query)
    print(f"Query type classified as: {query_type}")

    enhanced_query = enhance_query(user_query)
    print(f"Enhanced Query: {enhanced_query}")

    result = qa_chain({"query": enhanced_query})
    answer = result['result']
    print(f"Answer:\n{answer}")

    return answer

with gr.Blocks(title="✨Intelligent QA System with Feedback Loop🚀", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 🤖 Intelligent QA System 🤖

    Ask your questions, and let us know if the answer was helpful!  😊
    """)
    with gr.Row():
        user_query = gr.Textbox(label="🤔 Enter your query", placeholder="Type your question here...")
        submit = gr.Button("🚀 Submit Query")

    answer = gr.Textbox(label="✅ Answer", interactive=False)
    feedback_textbox = gr.Textbox(label="💬 Feedback", visible=False)
    reason_textbox = gr.Textbox(label="📝 Reason", visible=False, interactive=True)
    submit_reason_button = gr.Button("✅ Submit Reason", visible=False)  # Add a "Submit Reason" button

    with gr.Row():
        feedback_button_yes = gr.Button("👍 Yes (Helpful)")
        feedback_button_no = gr.Button("👎 No (Not Helpful)")

    def submit_event(user_query):
        return run_feedback_loop_simulation(user_query), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    submit.click(fn=submit_event, inputs=user_query, outputs=[answer, feedback_textbox, reason_textbox, submit_reason_button])

    def handle_feedback(answer, feedback_val):
        if feedback_val == "👍 Yes (Helpful)":
            return "Thank you for your feedback! 😊", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
        else:
            return "Could you please provide a reason? 🤔", gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)

    feedback_button_yes.click(fn=lambda answer: handle_feedback(answer, "👍 Yes (Helpful)"), inputs=answer, outputs=[feedback_textbox, reason_textbox, submit_reason_button])
    feedback_button_no.click(fn=lambda answer: handle_feedback(answer, "👎 No (Not Helpful)"), inputs=answer, outputs=[feedback_textbox, reason_textbox, submit_reason_button])

    def submit_reason(reason):
        print(f"User reason: {reason}")
        return "Thank you for your feedback! We appreciate your input. 🙏", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    submit_reason_button.click(fn=submit_reason, inputs=reason_textbox, outputs=[feedback_textbox, reason_textbox, submit_reason_button])

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6cd9ecf24da35747fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
